# HW2
### Minju Jo

## Q1

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100


#train과 test set으로 나누어 MNIST data 불러오기

mnist_train = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(),
                        download = True)
mnist_test = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor(),
                       download = True)


#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                         batch_size=batch_size,
                         shuffle=False,
                         drop_last=True)

In [4]:
import torch.nn as nn

In [5]:
for x, y in train_loader:
    print(x.shape)
    break

torch.Size([100, 1, 28, 28])


In [6]:
# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

input_size = 1*28*28
output_size = 10 # one to ten

l1 = nn.Linear(input_size, 100, bias = True)
l2 = nn.Linear(100, 100, bias = True)
l3 = nn.Linear(100, output_size, bias = True)
dropout = nn.Dropout(p = 0.3)
relu = nn.ReLU()
bn1 = nn.BatchNorm1d(100)
bn2 = nn.BatchNorm1d(100)

#xavier initialization을 이용하여 각 layer의 weight 초기화

nn.init.xavier_uniform_(l1.weight)
nn.init.xavier_uniform_(l2.weight)
nn.init.xavier_uniform_(l3.weight)

Parameter containing:
tensor([[-1.0766e-01, -5.1050e-02, -1.1665e-01,  1.3228e-01, -1.1840e-01,
          3.3563e-02, -1.0127e-01,  1.3994e-01,  1.4761e-01, -2.2170e-01,
          2.5772e-02, -2.1666e-02,  1.8241e-02,  5.0133e-02, -2.2530e-01,
         -2.2008e-01,  6.7254e-02,  1.1799e-01,  1.2317e-01,  3.9237e-02,
         -1.4996e-01,  1.1901e-02,  5.8836e-02, -1.7138e-01,  2.1557e-01,
         -1.9901e-01, -2.1536e-01,  5.0881e-02,  2.1389e-01, -2.6479e-02,
         -2.0792e-01,  1.8087e-01, -1.5557e-01, -6.8209e-02, -1.6699e-01,
         -8.6963e-02, -1.7932e-01,  1.8026e-01, -2.1254e-01, -6.1511e-02,
          1.2330e-01,  6.0720e-02,  1.4240e-02, -2.0966e-01, -7.0852e-03,
         -1.5043e-01,  3.0338e-02, -1.9515e-01, -1.1159e-01, -2.0493e-01,
          2.0340e-01, -6.3082e-02,  1.7558e-01,  2.2384e-03,  1.8260e-02,
          1.1779e-01, -1.7339e-01, -5.3052e-02,  1.6983e-01, -2.0256e-01,
         -4.2403e-02, -6.7203e-02,  1.0743e-01,  9.9188e-02, -2.1178e-01,
          7.0026

In [7]:
# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model = nn.Sequential(l1, bn1, relu, dropout,
                      l2, bn2, relu, dropout,
                      l3).to(device)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)

criterion = nn.CrossEntropyLoss().to(device)

#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
#cost 계산을 위한 변수 설정

model.train()
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 

for epoch in range(training_epochs):
    avg_cost = 0

    
#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        # reshape input image and label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        train_pred = model(X)
        bn_loss = criterion(train_pred, Y)
        
        optimizer.zero_grad()
        bn_loss.backward()
        optimizer.step()
        
        avg_cost += bn_loss / train_total_batch
        
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.508165479
Epoch: 0002 cost = 0.375495285
Epoch: 0003 cost = 0.333090812
Epoch: 0004 cost = 0.325469553
Epoch: 0005 cost = 0.298735440
Epoch: 0006 cost = 0.294380337
Epoch: 0007 cost = 0.271437466
Epoch: 0008 cost = 0.266944289
Epoch: 0009 cost = 0.267397463
Epoch: 0010 cost = 0.263606280
Epoch: 0011 cost = 0.252812713
Epoch: 0012 cost = 0.249431238
Epoch: 0013 cost = 0.247606575
Epoch: 0014 cost = 0.233987093
Epoch: 0015 cost = 0.241846159
Learning finished


In [9]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    
    model.eval()
    bn_loss = 0
    bn_acc = 0
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    pred = model(X_test)
    correct = torch.argmax(pred, 1) == Y_test
    accuracy = correct.float().mean()
    
    print("Accuracy: ", accuracy.item())
    
    
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy:  0.9362000226974487
Label:  8
Prediction:  8


/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## Q2

In [10]:
def train(model, optimizer):
    #Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
    model.train()
    
    for epoch in range(training_epochs):
        avg_cost = 0


    #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
        for X, Y in train_loader:
            # reshape input image and label is not one-hot encoded
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            train_pred = model(X)
            bn_loss = criterion(train_pred, Y)
            bn_loss.backward()
            optimizer.step()

            avg_cost += bn_loss / train_total_batch


        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

    print('Learning finished')

In [11]:
def test(model):
    #test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
    #X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
    #accuracy의 초기 값 설정(0으로) 꼭 할 것

    with torch.no_grad():
        
        model.eval()
        bn_loss = 0
        bn_acc = 0

        X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
        Y_test = mnist_test.test_labels.to(device)

        pred = model(X_test)
        correct = torch.argmax(pred, 1) == Y_test
        accuracy = correct.float().mean()

        print("Accuracy: ", accuracy.item())



        ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
        r = random.randint(0, len(test_loader)-1)
        X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
        Y_single_data = mnist_test.test_labels[r:r + 1]

        print('Label: ', Y_single_data.item())
        single_prediction = model(X_single_data)
        print('Prediction: ', torch.argmax(single_prediction, 1).item())

### bn1 : 200, bn2 : 100

In [12]:
l1 = nn.Linear(input_size, 200, bias = True)
l2 = nn.Linear(200, 100, bias = True)
l3 = nn.Linear(100, output_size, bias = True)
dropout = nn.Dropout(p = 0.3)
relu = nn.ReLU()
bn1 = nn.BatchNorm1d(200)
bn2 = nn.BatchNorm1d(100)

#xavier initialization을 이용하여 각 layer의 weight 초기화

nn.init.xavier_uniform_(l1.weight)
nn.init.xavier_uniform_(l2.weight)
nn.init.xavier_uniform_(l3.weight)

model = nn.Sequential(l1, bn1, relu, dropout,
                      l2, bn2, relu, dropout,
                      l3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
train(model, optimizer)

Epoch: 0001 cost = 0.457052320
Epoch: 0002 cost = 0.347828001
Epoch: 0003 cost = 0.308138520
Epoch: 0004 cost = 0.280457348
Epoch: 0005 cost = 0.261511773
Epoch: 0006 cost = 0.245762274
Epoch: 0007 cost = 0.236824229
Epoch: 0008 cost = 0.236409068
Epoch: 0009 cost = 0.233064726
Epoch: 0010 cost = 0.221178889
Epoch: 0011 cost = 0.216698870
Epoch: 0012 cost = 0.212435573
Epoch: 0013 cost = 0.214893490
Epoch: 0014 cost = 0.196609765
Epoch: 0015 cost = 0.195929185
Learning finished


In [14]:
test(model)

Accuracy:  0.8988000154495239
Label:  6
Prediction:  6


### bn1 : 100, bn2 : 50

In [15]:
l1 = nn.Linear(input_size, 100, bias = True)
l2 = nn.Linear(100, 50, bias = True)
l3 = nn.Linear(50, output_size, bias = True)
dropout = nn.Dropout(p = 0.3)
relu = nn.ReLU()
bn1 = nn.BatchNorm1d(100)
bn2 = nn.BatchNorm1d(50)

#xavier initialization을 이용하여 각 layer의 weight 초기화

nn.init.xavier_uniform_(l1.weight)
nn.init.xavier_uniform_(l2.weight)
nn.init.xavier_uniform_(l3.weight)

model = nn.Sequential(l1, bn1, relu, dropout,
                      l2, bn2, relu, dropout,
                      l3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [16]:
train(model, optimizer)

Epoch: 0001 cost = 0.495852977
Epoch: 0002 cost = 0.357113242
Epoch: 0003 cost = 0.326630712
Epoch: 0004 cost = 0.311622858
Epoch: 0005 cost = 0.277846158
Epoch: 0006 cost = 0.286495715
Epoch: 0007 cost = 0.275808871
Epoch: 0008 cost = 0.261058152
Epoch: 0009 cost = 0.256663710
Epoch: 0010 cost = 0.249630675
Epoch: 0011 cost = 0.250210762
Epoch: 0012 cost = 0.239698932
Epoch: 0013 cost = 0.248992592
Epoch: 0014 cost = 0.236774951
Epoch: 0015 cost = 0.240820155
Learning finished


In [17]:
test(model)

Accuracy:  0.9537000060081482
Label:  3
Prediction:  3
